In [10]:
import os
import sys

import numpy as np
import pandas as pd
import scipy.io

import mypd

root_folder = '/home/diego/storage/projects/auditory-cortex/data/'

In [11]:
# spikes_mat['param']['nsamples']

In [14]:
neuron_folder = 'ACx_data_3/ACxCalyx/20200717-xxx999-002-001'
# neuron_folder = 'ACx_data_3/ACxCalyx/20200718-xxx999-001-002'
folder = root_folder + neuron_folder

list_files = os.listdir(root_folder + neuron_folder)
stim_file, spikes_file = None, None

for f in list_files:
    if 'stimuli' in f:
        stim_file = f
    elif 'tt_spikes' in f:
        spikes_file = f
    elif stim_file is not None and spikes_file is not None:
        break

stimuli_mat = scipy.io.loadmat(folder + '/' + stim_file)
spikes_mat = scipy.io.loadmat(folder + '/' + spikes_file)

arg_spikes = spikes_mat['timestamps']
dt = 1 / int(spikes_mat['param']['samplerate'][0]) * 1000
n_samples = spikes_mat['param']['nsamples']
tf = float(n_samples * dt)

if len(stimuli_mat['stimuli']) > 0:

    stim_type = np.array([stimuli_mat['stimuli']['type'][ii][0][0] \
                          for ii in range(len(stimuli_mat['stimuli']['type']))])

    stimlength = np.array([float(stimuli_mat['stimuli']['stimlength'][ii]) \
                           for ii in range(len(stimuli_mat['stimuli']['stimlength']))])

    trigger = np.array([int(stimuli_mat['stimuli']['trigger'][ii]) \
                        for ii in range(len(stimuli_mat['stimuli']['trigger']))]) * dt

    tone_params = ['amplitude', 'ramp', 'frequency']
    sweep_params = ['start_frequency', 'stop_frequency', 'speed', 'method', 'next']
    pulse_params = ['start', 'width', 'height', 'npulses', 'isi']
    #         wn_params = ['amplitude', 'duration', 'ramp']
    click_params = ['clickduration', 'nclicks']

    df = pd.DataFrame(np.stack((stim_type, trigger, stimlength), axis=1), 
                           columns=['type', 'trigger', 'duration'])
    df['trigger'] = df['trigger'].astype(float)

    for col in tone_params + sweep_params + pulse_params + click_params:
        df[col] = np.nan
    #         display(self.df)
    for ii, idx in enumerate(df.index):
        cols = list(stimuli_mat['stimuli'][ii][0][1].dtype.fields.keys())
        cols.remove('duration')
        vals = [stimuli_mat['stimuli'][ii][0][1][f][0][0][0][0] for f in cols]
        df.loc[idx, cols] = vals
    df['method'] = df['method'].astype(str)
    df.loc[df['method'] == 'l', 'method'] = 'logarithmic'

    df['speed'] =  df['speed'].abs()
    df['sweep_direction'] = df['stop_frequency'] - df['start_frequency']
    df['sweep_direction'] = df['sweep_direction'].map(lambda x: 'up' if x > 0 else 'down' if x < 0 else 'NaN')

    df['speed_abs'] = df['speed'].abs().copy()
    df['speed'] = df['speed'] * df['sweep_direction'].map(dict(up=1, down=-1))
    df[['speed', 'speed_abs']] = mypd.float_to_int(df.copy(), columns=['speed', 'speed_abs'])
    df['duration'] = df['duration'].astype(float)

In [15]:
df.loc[df['method'] == 'l', 'method'] = 'logarithmic'

In [16]:
stimuli_mat['stimuli'][0][0][1]#['']

array([[(array([[10]], dtype=uint8), array([[5]], dtype=uint8), array([[69]], dtype=uint8), array([[102]], dtype=uint8), array([[0]], dtype=uint8), array([[5]], dtype=uint8), array([[1]], dtype=uint8), array([[3000]], dtype=uint16))]],
      dtype=[('nclicks', 'O'), ('clickduration', 'O'), ('isi', 'O'), ('amplitude', 'O'), ('start', 'O'), ('duration', 'O'), ('ramp', 'O'), ('next', 'O')])

In [19]:
df['end_stimulus'] = df['trigger'] + df['duration']
df['next_trigger'] = np.append(df.iloc[1:]['trigger'].values, tf)
df['after_stimulus_period'] = df['next_trigger'] - df['end_stimulus']

In [20]:
df['after_stimulus_period'].describe()

count     116.000000
mean     2077.971552
std       774.334827
min       851.100000
25%      1700.100000
50%      2103.550000
75%      2650.325000
max      2867.200000
Name: after_stimulus_period, dtype: float64